In [1]:
import joblib
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


In [2]:
# Cargar los modelos desde los ficheros
best_rf_regressor = joblib.load('best_rf_regressor.pkl')
best_xgboost_regressor = joblib.load('best_xgboost_regressor.pkl')
best_mlr_regressor = joblib.load('best_mlr_regressor.pkl')

In [3]:
file = open('factores.txt','r')

factores = []
for i in file:
    print(i)
    factores.append(i.replace('\n',''))

factor_rf = float(factores[0])
factor_mlr = float(factores[1])
factor_xgboost = float(factores[2])
total_mse = float(factores[3])

0.24726642447520827

0.5901025163285543

0.16263105919623744

0.09816154345891576


In [4]:
total_mse

0.09816154345891576

In [5]:
df = pd.read_excel('predictions_years.xlsx')

In [6]:
df

,Fecha,VIX,Unemployment,Interest_Rates,YIC,M2 MONEY STOCK,PER CAPITA CONSUMPTION,CONSUMER PRICE INDEX
0,2025-06-01,0.765318,0.885070,-0.359264,0.826913,1.224544,1.053485,1.094610
1,2026-06-01,0.667702,1.310944,-1.259594,1.599933,1.567947,1.148476,1.167403
2,2027-06-01,0.233381,0.993995,-0.797966,1.362266,1.666989,1.299526,1.328394


In [7]:
df = df.rename(columns={'Fecha': 'date'})

In [8]:
df

,date,VIX,Unemployment,Interest_Rates,YIC,M2 MONEY STOCK,PER CAPITA CONSUMPTION,CONSUMER PRICE INDEX
0,2025-06-01,0.765318,0.885070,-0.359264,0.826913,1.224544,1.053485,1.094610
1,2026-06-01,0.667702,1.310944,-1.259594,1.599933,1.567947,1.148476,1.167403
2,2027-06-01,0.233381,0.993995,-0.797966,1.362266,1.666989,1.299526,1.328394


In [9]:
x_real = df.drop('date', axis = 1)
# y_real, y_date = df['USREC'], df['date']

rf_test_pred = best_rf_regressor.predict(x_real)
xgboost_test_pred = best_xgboost_regressor.predict(x_real)
mlr_test_pred = best_mlr_regressor.predict(x_real)

In [10]:


predictions = pd.DataFrame()

predictions['Random Forest'] = rf_test_pred
predictions['XG Boost'] = xgboost_test_pred
predictions['Multiple Linear Regression'] = mlr_test_pred

predictions['Probabilidad ponderada'] = (((predictions['Random Forest']/factor_rf) + (predictions['XG Boost']/factor_xgboost) + (predictions['Multiple Linear Regression']/factor_mlr))*total_mse).round(2)

# # Obtener todas las columnas excepto las dos primeras ('date' y 'USREC')
# columns_to_scale = predictions.columns[2:]  

# # Inicializar el objeto MinMaxScaler
# scaler = MinMaxScaler()

# # Aplicar el escalado Min-Max a las columnas seleccionadas
# predictions[columns_to_scale] = scaler.fit_transform(predictions[columns_to_scale])

# Aplicar el escalado Min-Max a la columna 'probabilidad'
predictions['Probabilidad ponderada'] = predictions['Probabilidad ponderada']*100

In [11]:
predictions

,Random Forest,XG Boost,Multiple Linear Regression,Probabilidad ponderada
0,0.693333,0.421857,0.683319,64.0
1,0.680000,0.434881,0.368392,59.0
2,0.046667,0.023579,0.107532,5.0


In [12]:
print(rf_test_pred)
print(xgboost_test_pred)
print(mlr_test_pred)

[0.69333333 0.68       0.04666667]
[0.42185688 0.4348811  0.02357912]
[0.68331899 0.36839203 0.107532  ]
